In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-sample.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-labels.pkl
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-wav.tar

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import malaya_speech
malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.stream` is not able to use.


<module 'malaya_speech' from '/home/husein/dev/malaya-speech/malaya_speech/__init__.py'>

In [4]:
model = malaya_speech.speaker_vector.huggingface('microsoft/unispeech-sat-base-plus-sv')

In [5]:
_ = model.cuda()

In [6]:
import json
import pickle
from tqdm import tqdm

In [7]:
with open('voxceleb2-test-sample.json') as fopen:
    sample_files = json.load(fopen)

In [8]:
sample_files[0]

'/home/husein/youtube/voxceleb-wav/aac-id08456-gWxUPq1QZlI-00257.wav'

In [9]:
with open('voxceleb2-test-labels.pkl', 'rb') as fopen:
    labels = pickle.load(fopen)

In [10]:
unique_files = []
for l in labels:
    unique_files.extend(l[1:])
    
unique_files = list(set(unique_files))

In [11]:
len(unique_files)

36237

In [12]:
import torch

In [24]:
vectors = {}
for f in tqdm(unique_files):
    y_, _ = malaya_speech.load(f.replace('/home/husein/youtube/', '/home/husein/'))
    try:
        v = model([y_])[0]
        vectors[f] = v
    except:
        torch.cuda.empty_cache()

100%|█████████████████████████████████████| 36237/36237 [09:25<00:00, 64.05it/s]


In [30]:
import numpy as np

scores, ls = [], []

for i in tqdm(range(len(labels))):
    if labels[i][1] in vectors and labels[i][2] in vectors:
        ls.append(labels[i][0])
        scores.append(np.sum(vectors[labels[i][1]] * vectors[labels[i][2]]))

100%|█████████████████████████████| 5900000/5900000 [00:16<00:00, 356812.13it/s]


In [31]:
len(scores), len(vectors)

(5899877, 36236)

In [32]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [33]:
calculate_eer(ls, scores)

(0.06688427572303626, array(0.90226115))